In [ ]:

import librosa
import numpy as np
import torch
import IPython
from music2latent4 import Inferencer
import soundfile as sf

# Initialize the Inferencer
encdec = Inferencer()

# Load an example audio file (or use your own!)
#audio_path = librosa.example('trumpet')
audio_path="/home/eloi/projects/project_mfm_eloi/src/audio_examples/9220_ld_multi.wav"
wv, sr=sf.read(audio_path, dtype='float32')
#wv, sr = librosa.load(audio_path, sr=44100)  # Ensure 44.1kHz sample rate
wv=torch.from_numpy(wv).permute(1,0)  # Add batch dimension

print(wv.shape)
length=44100*5
start=15*sr
end=start+length
wv=wv[...,start:end]  # Truncate to 5 seconds for faster processing


print("wv", wv.shape)

# Encode the audio
latent = encdec.encode(wv)

def latent2seq(latent):
    """
    Convert the latent representation to a sequence of latent vectors.
    """
    # Reshape the latent representation to match the expected input shape
    latent = latent.view(latent.size(0), -1)
    return latent

def seq2latent(latent_sequence):
    """
    Convert the sequence of latent vectors back to the original latent representation.
    """
    # Reshape the latent sequence to match the expected output shape
    latent = latent_sequence.view(latent_sequence.size(0), -1, 64)
    return latent


latent=latent2seq(latent)
print("latent", latent.shape)

# Decode the latent representation back to a waveform
lateent=seq2latent(latent)
wv_rec = encdec.decode(latent)

print("wv_rec", wv_rec.shape)

# Listen to the original and reconstructed audio
print('Original')
IPython.display.display(IPython.display.Audio(wv, rate=sr))
print('Reconstructed')
IPython.display.display(IPython.display.Audio(wv_rec.squeeze().cpu().numpy(), rate=sr))

torch.Size([2, 11065214])
wv torch.Size([2, 262144])
latent torch.Size([8, 8, 64])
latent_sequence torch.Size([8, 512])
wv_rec torch.Size([2, 263168])
Original


Reconstructed
